## Famous personalities in history on wikipedia

#### Outline
- we will scrape through the website wikipedia , url - https://en.wikipedia.org/wiki/Wikipedia:Vital_articles
- we will get a list of people, for each person we will describe there personal details like born, died, area, description, image-url etc.
- convert it to csv file.



In [113]:
! pip install requests --quiet
! pip install pandas --quiet
! pip install beautifulsoup4 --upgrade --quiet
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [114]:

resource_url = "https://en.wikipedia.org/wiki/Wikipedia:Vital_articles"

response = requests.get(resource_url)
print(response.status_code)
len(response.text) # chars

200


946615

In [115]:
content = response.text
content[:100]

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-la'

In [117]:
doc = BeautifulSoup(content, 'html.parser')
type(doc)

bs4.BeautifulSoup

In [118]:
tags = doc.find_all('table', {'class':'multicol'})
a_tags = tags[0].find_all('a', _class="")

names = []
for tag in a_tags:
    if tag.text !="":
        names.append(tag.text)

names[:5]

['Hammurabi',
 'Hatshepsut',
 'Ramesses II',
 'Cyrus the Great',
 'Alexander the Great']

In [119]:
people_url = []

for tag in a_tags:
    if tag.text !="":
        people_url.append("https://en.wikipedia.org"+tag['href'])

people_url[:5]

['https://en.wikipedia.org/wiki/Hammurabi',
 'https://en.wikipedia.org/wiki/Hatshepsut',
 'https://en.wikipedia.org/wiki/Ramesses_II',
 'https://en.wikipedia.org/wiki/Cyrus_the_Great',
 'https://en.wikipedia.org/wiki/Alexander_the_Great']

In [120]:
people_df = pd.DataFrame({'names':names, 'url':people_url})
people_df.head()

,names,url
0,Hammurabi,https://en.wikipedia.org/wiki/Hammurabi
1,Hatshepsut,https://en.wikipedia.org/wiki/Hatshepsut
2,Ramesses II,https://en.wikipedia.org/wiki/Ramesses_II
3,Cyrus the Great,https://en.wikipedia.org/wiki/Cyrus_the_Great
4,Alexander the Great,https://en.wikipedia.org/wiki/Alexander_the_Great


In [122]:
def get_page(detail_url):
    response = requests.get(detail_url)
    detail_doc = BeautifulSoup(response.text, 'html.parser')
    return detail_doc

In [123]:

def get_description(detail_doc):
    table_tag = detail_doc.find_all('table')
    first_table = table_tag[0]
    tr_tags = first_table.find_all('tr')
    description = ""
    for tr_tag in tr_tags:
        th = tr_tag.find('th')
        td = tr_tag.find('td')
        if th != None and td != None:
            description += str(th.text) + " : " +  str(td.text) + ", "

    return description


In [139]:
def get_image(detail_doc):
    td_tag = detail_doc.find('td', {'class':"infobox-image"})
    if td_tag != None:
        a_tag = td_tag.find('a')
        if a_tag != None:
            image_url = "https://en.wikipedia.org"+ a_tag['href']
    else :
        image_url = ""
    return image_url


In [141]:
description_list = []
image_url_list = []
for index, row in people_df.iterrows():
    detail_url = row["url"]
    detail_doc = get_page(detail_url)
    description_list.append(get_description(detail_doc))
    image_url_list.append(get_image(detail_doc))

In [143]:
people_df['image'] = image_url_list
people_df['description'] = description_list
people_df.to_csv('famous_personalities.csv', index=None)


In [158]:
people_df[0:5]

,names,url,image,description
0,Hammurabi,https://en.wikipedia.org/wiki/Hammurabi,https://en.wikipedia.org/wiki/File:F0182_Louvr...,"Reign : c. 1792 – c. 1750 BC (MC), Predecessor..."
1,Hatshepsut,https://en.wikipedia.org/wiki/Hatshepsut,https://en.wikipedia.org/wiki/File:Seated_Stat...,"Reign : c. 1478/9 – 1458 BC, Predecessor : Thu..."
2,Ramesses II,https://en.wikipedia.org/wiki/Ramesses_II,https://en.wikipedia.org/wiki/File:Ramses_II_B...,"Reign : 1279–1213 BC, Predecessor : Seti I, Su..."
3,Cyrus the Great,https://en.wikipedia.org/wiki/Cyrus_the_Great,https://en.wikipedia.org/wiki/File:Cyrus_II_(T...,"Reign : 559–530 BC, Predecessor : Empire estab..."
4,Alexander the Great,https://en.wikipedia.org/wiki/Alexander_the_Great,https://en.wikipedia.org/wiki/File:Alexander_t...,"Reign : 336–323 BC, Predecessor : Philip II, S..."
